# Анализ поведения клиентов.

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Нам необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных уже проведена.
Необходимо построить модель с максимально большим значением accuracy. Чтобы считать проект успешным, нужно довести долю правильных ответов по крайней мере до 0.75. Далее проверим значение accuracy на тестовой выборке.

**Описание данных:** \
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

**Известно:** \
**сalls** — количество звонков,\
**minutes** — суммарная длительность звонков в минутах,\
**messages** — количество sms-сообщений,\
**mb_used** — израсходованный интернет-трафик в Мб,\
**is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

**Цель исследования:** Необходимо построить модель для задачи классификации, которая выберет подходящий тариф и построить модель с максимально большим значением accuracy. Необходимо довести долю правильных ответов по крайней мере до 0.75 и проверить значение accuracy на тестовой выборке.

**Ход исследоввания:** Изучим файлы с данными users_behavior.csv. Разделим исходные данные на обучающую, валидационную и тестовую выборки. Исследуем качество разных моделей меняя гиперпараметры. Проверим качество модели на тестовой выборке. Проверим модели на вменяемость.

**Исследование пройдет в шесть этапов:**

1) Изучение общей информации о данных.
2) Разделение исходных данных на выборки.
3) Исследование качества моделей при изменении гиперпараметров.
4) Проверка качества модели на тестовой выборке.
5) Проверка модели на вменяемость.
6) Общий вывод.


# 1. Откроем файл с данными и изучим общую информацию. 

Импортируем необходимые библиотеки. Считаем данные из csv-файла в датафрейм и сохраним в переменную data. Путь к файлам: /Users/Slava/Desktop/Яндекс Практикум учебные материалы/Проекты/Проект6.

In [1]:
# импортируем необходимые библиотеки
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats as st
import numpy as np
from IPython.display import display

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.dummy import DummyClassifier

In [2]:
# прочитаем данные из csv-файла в датафрейм и сохраним в переменную 
try:
    data = pd.read_csv('/Users/Slava/Desktop/Яндекс Практикум учебные материалы/Проекты/Проект6./users_behavior.csv', sep = ',')
except: 
     data = pd.read_csv('/datasets/users_behavior.csv', sep = ',') 

In [3]:
# ознакомимся с общей информацией о данных
data.info()

Типы данных корректы.

In [4]:
# выведем первые 10 строк набора данных, ознакомимся с признаками и объектами
data.head(10)

In [5]:
# ознакомимся с описательной статистикой набора данных
data.describe()

In [6]:
# на всякий случай проверим пропуски в данных
data.isna().sum()

В данных 3214 объектов, пропуски отсутствуют. Исходя из постановки задачи, целевым признаком является - 'is_ultra'. Он принимает значения 0 или 1, а значит, перед нами классическая задача бинарной классификации.

Оценим соотношение пользователей в наборе данных с тарифом 'ultra' и 'smart'.

In [7]:
data['is_ultra'].value_counts()/len(data)

Значений - '1' в признаке 'is_ultra' порядка ~ 30%, что означает значения представлены неравномерно, что логично, как правило более продвинутым тарифом пользуется меньшее количество клиентов.

# 2. Разделение исходных данных на выборки.

In [8]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra'] 
features.shape, target.shape

Разделим выборку на обучающую, валидационную и тестовую в отношении 3:1:1.\
Так же мы учтем неравномерность представления целевых признаков и используем параметр 'stratify'.

In [9]:
# сначала разобъем выборку на обучающую и 'осаточную' в пропорции 60/40,
# применим 'stratify' для равномерного разделения значений в признаке.
features_train, features_remaining, target_train, target_remaining = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)
# разобъем 'остаточную' выборку на тестовую и валидационную в пропорциях 50/50, 
# так же применив 'stratify' для равномерного разделения значений в 'остаточной' выборке.
features_valid, features_test, target_valid, target_test = train_test_split(
    features_remaining, target_remaining, test_size=0.5, random_state=12345, stratify=target_remaining)

print(f'Размеры выборок:')
print(f'Обучающая: {len(features_train)}')
print(f'Валидационная: {len(features_valid)}')
print(f'Тестовая: {len(features_test)}')

In [10]:
# проверим сохранение пропорций равномерного распределения значений в выборках
for tr in [target_train, target_valid, target_test]:
    print(tr.value_counts()/len(tr))

Исходные данные разделены на обучающую, валидационную и тестовую в отношении 3:1:1. С помощью метода 'stratify' соблюдены пропорции соотношения целевых признаков. 

Данные готовы к исследованию.

# 3. Исследование качества моделей при изменении гиперпараметров.

Сравним по метрике accuracy модели: DecisionTreeClassifier, RandomForestClassifier, LogisticRegression с различными гиперпараметрами.

## 3.1 Модель DecisionTreeClassifier.

Создадим многоуровневый цикл перебирающий гиперпараметры применяемые к модели.

In [11]:
# для удобства визуализации результатов создадим табличку, 
# которая выведет 5 наиболее оптимальных гиперпараметров для заданной модели 
columns = ['criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'accuracy_score']
data = []
for criterion in ['gini', 'entropy']:
    for max_depth in range(1, 21):
        for min_samples_split in [0.01, 0.1]:
            for min_samples_leaf in [0.0001, 0.001]:
                model = DecisionTreeClassifier(criterion=criterion,
                        max_depth=max_depth,
                        min_samples_split=min_samples_split,
                        min_samples_leaf=min_samples_leaf,
                        random_state=12345)
                
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid)
                 
                score = accuracy_score(predictions, target_valid)
                string = [criterion, max_depth, min_samples_split, min_samples_leaf,
                         score]
                data.append(string)

In [12]:
# посмотрим 5 лучших результатов работы модели
tree_top_parameter = pd.DataFrame(columns=columns, data=data)
print('5 лучших наборов гиперпараметров')
tree_top_parameter.sort_values('accuracy_score', ascending = False).head()

In [13]:
# посмотрим 5 худших результатов работы модели
tree_top_parameter = pd.DataFrame(columns=columns, data=data)
print('5 худших наборов гиперпараметров')
tree_top_parameter.sort_values('accuracy_score', ascending = True).head()

В наборах гиперпараметров использовались критерии: entropy и gini. Исходя из результатов, критерий энтропии оказывает наибольшее положительное влияние на лучшие наборы модели.

При измерениях accuracy на валидационной выборке наилучшей максимальной глубиной явно является глубина, равная 9. Начиная со значения 10 и дальнейшее увеличение глубины дерева приводит к переобучению модели. Слишком маленькая глубина, менее 4, не позволила получить удовлетворительные результаты.\
Стоит обратить внимание, что min_samples_split и min_samples_leaf в данном случае практически равнозначно положительно повлияли на лучшие наборы модели.

Подводя итоги применения данной модели, можно утверждать, что при решении поставленной задачи наиболее важным гиперпарамтром является максимальная глубина дерева, так как ее изменения в корне меняют конечный результат.

Сохраним лучший результат.


In [14]:
best_tree = tree_top_parameter.sort_values('accuracy_score', ascending = False).head(1)
best_tree

## 3.2 Модель RandomForestClassifier.

Так же создадим многоуровневый цикл перебирающий гиперпараметры применяемые к модели.

In [15]:
columns = ['n_estimators', 'class_weight', 'bootstrap', 'accuracy_score']
data = []
for n_estimators in range(1, 201, 20):
    for class_weight in ['balanced', None]:
        for bootstrap in [True, False]:
            for min_samples_leaf in [0.0001, 0.001, 0.01]:
                model = RandomForestClassifier(n_estimators=n_estimators,
                        class_weight=class_weight,
                        bootstrap=bootstrap,
                        random_state=12345)
                
                model.fit(features_train, target_train)
                predictions = model.predict(features_valid)
               
                score = accuracy_score(predictions, target_valid)
                string = [n_estimators, class_weight, bootstrap,
                         score]
                data.append(string)

In [16]:
# посмотрим 5 лучших результатов работы модели
forest_top_parameter = pd.DataFrame(columns=columns, data=data)
print('5 лучших наборов гиперпараметров')
forest_top_parameter.sort_values('accuracy_score', ascending = False).head()

In [17]:
# посмотрим 5 худших результатов работы модели
forest_top_parameter = pd.DataFrame(columns=columns, data=data)
print('5 худших наборов гиперпараметров')
forest_top_parameter.sort_values('accuracy_score', ascending = True).head()

Качество модели RandomForestClassifier намного меньше зависит от подбора гиперпараметров, чем качество модели DecisionTreeClassifier.
Как и ожидалось, качество модели случайного леса выше, однако скорость выполнения расчета низкая.

Из результатов видно, что существенное влияние оказывает гиперпараметр bootstrap: так как во всех 'лучших' моделях он принимает значение True, однако и в худших присутствует.

Бесконечное увеличение количества деревьев не приводит к улучшению качества модели: она переобучется. У лучшей модели не самое большое число n_estimators.\
Существенно улучшает 5 лучших наборов гиперпараметр минимального количества объектов в листе.

Сохраним лучший результат модели случайного леса. 

In [18]:
best_rf = forest_top_parameter.sort_values('accuracy_score', ascending = False).head(1)
best_rf

## 3.3 Модель LogisticRegression.

Создадим модель логистической регрессии.

In [19]:
model = LogisticRegression(max_iter=1000,
        solver='lbfgs',
        random_state=12345)
                
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
               
regression_score = accuracy_score(predictions, target_valid)

In [20]:
# посмотрим результат работы модели логистической регрессии
print(f'Значение accuracy модели логистической регрессии:', regression_score)

Качество модели логистическая регрессия на валидационной выборке ниже, чем и у модели дерево решений, и у модели случайный лес.

При работе с данной моделью были сделаны выводы, что изменение гиперпараметров модели, в значительной степени не влияют на качество предсказания модели. Влияние слабое и более разбросано. В коненчном итоге, результат либо приемлемый, либо плохой. Можно сделать вывод, что модель имеет тонкую грань к переобучению.

# 4. Проверка качества модели на тестовой выборке.

Выведем лучшие наборы гиперпараметров для каждого семейства моделей и соответсвующие им accuracy_score.

In [22]:
total_results = pd.DataFrame({'Accuracy DecisionTreeClassifier': ['0.802'], 'Accuracy RandomForestClassifier' : ['0.811'], 'Accuracy LogisticRegression': ['0.738']})
total_results

Таким образом, наибольшего accuracy_score удалось добиться с помощью модели случайного леса (RandomForestClassifier) у дерева решений (DecisionTreeClassifier) так же корректный и достаточно близкий результат. Видимо, в связи с тем, что зависимость ответа от параметров не 'достаточно' линейная, модель регрессии использовать для данной выборки не совсем целесообразно.

Проверим качество лучшей модели с заданными гиперпараметрами на тестовой выборке.

In [23]:
model = RandomForestClassifier(n_estimators=61,
        class_weight=None,
        bootstrap=True,
        random_state=12345)

model.fit(features_train, target_train)
score = model.score(features_test, target_test)
print('accuracy_score на тестовой выборке:', score)

На тестовой выборке accuracy_score оказался немного ниже, чем на валидационной, однако при увеличении числа деревьев точность увеличивается. Тем не менее, требование, указанное в задании (accuracy_score > 0.75) выполнено.

# 5. Проверка модели на вменяемость.

При обучении с учителем простая проверка работоспособности состоит из сравнения своей оценки с простыми практическими правилами. DummyClassifier реализует несколько таких простых стратегий классификации:

**stratified** - генерирует случайные прогнозы, соблюдая распределение классов обучающего набора.\
**most_frequent** - всегда предсказывает наиболее частую метку в обучающем наборе.\
**prior** - всегда предсказывает класс, который максимизирует предыдущий класс (как most_frequent) и predict_proba возвращает предыдущий класс.\
**uniform** - генерирует предсказания равномерно в случайном порядке.\
**constant** - всегда предсказывает постоянную метку, предоставленную пользователем. Основная мотивация этого метода — оценка F1, когда положительный класс находится в меньшинстве.

Применим данный метод сравнения полученной оценки.

In [24]:
print("accuracy_score на тестовых данных:", score)

dummy_clf = DummyClassifier(strategy="uniform", random_state = 12)
dummy_clf.fit(features_train, target_train)
print("accuracy_score на случайных 0 и 1:", accuracy_score(dummy_clf.predict(features_test), target_test))

dummy_clf = DummyClassifier(strategy="stratified", random_state = 12)
dummy_clf.fit(features_train, target_train)
print("accuracy_score на стратифицированной выборке:", accuracy_score(dummy_clf.predict(features_test), target_test))

dummy_clf = DummyClassifier(strategy="most_frequent", random_state = 12)
dummy_clf.fit(features_train, target_train)
print("accuracy_score на наиболее частых значениях:", accuracy_score(dummy_clf.predict(features_test), target_test))

dummy_clf = DummyClassifier(strategy="constant", random_state = 12, constant = 0)
dummy_clf.fit(features_train, target_train)
print("accuracy_score на нулях:", accuracy_score(dummy_clf.predict(features_test), target_test))

dummy_clf = DummyClassifier(strategy="constant", random_state = 12, constant = 1)
dummy_clf.fit(features_train, target_train)
print("accuracy_score на единицах:", accuracy_score(dummy_clf.predict(features_test), target_test))

Обученная модель показывает себя лучше, чем все рассмотренные примтивы, а значит ее можно назвать адекватной.

# 6. Общий вывод.

По результатам исследования можно сделать следующие выводы:

Проведено исследование в целях разработки модели, позволяющей наиболее точно предложить пользователям мобильной связи тарифы «Смарт» или «Ультра».\
Модель построена на основании поведения клиентов, уже перешедших на эти тарифы.

Были построены 3 модели:

1) Дерево решений (Decision Tree Classifier).
2) Cлучайный лес (RandomForestClassifier).
3) Логистическая регрессия (Logistic Regression).
   
По результатам сравнения была выбрана лучшая - 'случайный лес' со следующими гиперпараметрами n_estimators=61, class_weight=None, bootstrap=True. Лучшее значение accuracy_score = 0.811.

Из всех отработанных моделей только модель логистической регрессии не соответствуют требованию по качеству, согласно которому значение accuracy должно быть не менее 0.75.
Лучшая модель проверена на тестовой выборке и получено значение accuracy = 0.8. 
Для проверки на адекватность нашей лучшей модели - случайный лес (RandomForestClassifier) была использована модель DummyClassifier, которая показала результат accuracy ниже, чем результат модели полученный в процессе исследования.

Результаты исследования позволят специалистам использовать полученную нами модель, которая подберет клиентам пользующимся архивными тарифами -  новый. 